In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

# =============================================================================
# 1. VERİ YÜKLEME VE TEMİZLEME (DÜZELTİLMİŞ KISIM)
# =============================================================================
def load_and_process_data():
    print("Veriler Yükleniyor ve İşleniyor...")

    # --- DÜZELTME BURADA YAPILDI ---
    # encoding='latin-1' eklendi. Eğer yine hata verirse 'cp1252' deneyebilirsin.
    try:
        places_df = pd.read_csv('paris_1000_mixed_places.csv', sep=';', encoding='latin-1')
        users_df = pd.read_csv('users.csv', sep=';', encoding='latin-1')
    except UnicodeDecodeError:
        # latin-1 de çalışmazsa cp1252 dene
        print("Latin-1 başarısız, cp1252 deneniyor...")
        places_df = pd.read_csv('paris_1000_mixed_places.csv', sep=';', encoding='cp1252')
        users_df = pd.read_csv('users.csv', sep=';', encoding='cp1252')

    # --- A. Mekan Verisi İşleme ---
    # Kategorileri temizle ve listeye çevir
    # NaN değerleri boş string yapıp string'e çeviriyoruz
    places_df['category'] = places_df['category'].fillna('').astype(str)

    # Virgülle ayrılmış kategorileri listeye çeviriyoruz
    places_df['category_list'] = places_df['category'].apply(lambda x: [i.strip().lower() for i in x.split(',') if i.strip()])

    # Mekanlar için özellik vektörü
    mlb_places = MultiLabelBinarizer()
    place_features = mlb_places.fit_transform(places_df['category_list'])

    # GNN Embeddinglerini Simüle Etme (Hedef Vektör Boyutu: 64)
    dummy_projection = np.random.rand(place_features.shape[1], 64)
    place_embeddings = np.dot(place_features, dummy_projection)
    place_embeddings = torch.tensor(place_embeddings, dtype=torch.float32)

    # --- B. Kullanıcı Verisi İşleme (GİRDİ) ---
    # İlgi alanlarını listeye çevir
    users_df['interests'] = users_df['interests'].fillna('').astype(str)
    users_df['interests_list'] = users_df['interests'].apply(lambda x: [i.strip().lower() for i in x.split(',') if i.strip()])

    # 1. İlgi Alanlarını Vektöre Çevir
    mlb_users = MultiLabelBinarizer()
    user_interest_features = mlb_users.fit_transform(users_df['interests_list'])

    # 2. Bütçe ve Yaş Grubunu Vektöre Çevir
    # Eğer budget veya age_group boşsa hata vermesin diye fillna yapıyoruz
    users_df['budget'] = users_df['budget'].fillna('Standard')
    users_df['age_group'] = users_df['age_group'].fillna('25-34')

    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    user_demographics = ohe.fit_transform(users_df[['budget', 'age_group']])

    # Tüm özellikleri birleştir
    user_features = np.hstack([user_interest_features, user_demographics])
    user_tensor = torch.tensor(user_features, dtype=torch.float32)

    # --- C. Hedef Oluşturma (Eğitim İçin) ---
    target_embeddings = []

    # MLB class isimlerini alalım (index eşleşmesi için)
    place_cats_list = list(mlb_places.classes_)

    for keywords in users_df['interests_list']:
        matched_indices = []

        # Hızlandırma için basit mantık:
        # Kullanıcının ilgi alanı kelimelerinden herhangi biri mekan kategorisinde geçiyor mu?
        for idx, cats in enumerate(places_df['category_list']):
            # Kesişim kümesi boş değilse eşleşme vardır
            if set(keywords) & set(cats):
                matched_indices.append(idx)

        if matched_indices:
            avg_emb = place_embeddings[matched_indices].mean(dim=0)
        else:
            avg_emb = torch.randn(64)
        target_embeddings.append(avg_emb)

    target_tensor = torch.stack(target_embeddings)

    return {
        'user_tensor': user_tensor,
        'target_tensor': target_tensor,
        'place_embeddings': place_embeddings,
        'places_df': places_df,
        'mlb_users': mlb_users,
        'ohe': ohe,
        'input_dim': user_tensor.shape[1],
        'embedding_dim': 64
    }

# =============================================================================
# 2. GAN MODELLERİ
# =============================================================================
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.2),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# =============================================================================
# 3. EĞİTİM FONKSİYONU
# =============================================================================
def train_coldgan(data_dict, epochs=300):
    user_input = data_dict['user_tensor']
    real_embeddings = data_dict['target_tensor']

    generator = Generator(data_dict['input_dim'], data_dict['embedding_dim'])
    discriminator = Discriminator(data_dict['embedding_dim'])

    opt_g = optim.Adam(generator.parameters(), lr=0.001)
    opt_d = optim.Adam(discriminator.parameters(), lr=0.001)
    criterion = nn.BCELoss()

    print(f"\nModel Eğitiliyor ({epochs} Epoch)...")

    for epoch in range(epochs):
        # --- Train Discriminator ---
        opt_d.zero_grad()
        real_preds = discriminator(real_embeddings)
        d_loss_real = criterion(real_preds, torch.ones_like(real_preds))

        fake_embeddings = generator(user_input)
        fake_preds = discriminator(fake_embeddings.detach())
        d_loss_fake = criterion(fake_preds, torch.zeros_like(fake_preds))

        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        opt_d.step()

        # --- Train Generator ---
        opt_g.zero_grad()
        fake_embeddings_2 = generator(user_input)
        preds_2 = discriminator(fake_embeddings_2)

        g_loss_adv = criterion(preds_2, torch.ones_like(preds_2))
        g_loss_content = nn.MSELoss()(fake_embeddings_2, real_embeddings)

        g_loss = g_loss_adv + (10 * g_loss_content)
        g_loss.backward()
        opt_g.step()

        if epoch % 50 == 0:
            print(f"Epoch {epoch} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

    print("Eğitim Tamamlandı!")
    return generator

# =============================================================================
# 4. YENİ KULLANICI TEST FONKSİYONU
# =============================================================================
def test_new_user(generator, data_dict, interests_str, budget_str, age_str):
    print(f"\n--- Yeni Kullanıcı Testi: {interests_str} | {budget_str} ---")

    interests_list = [i.strip().lower() for i in interests_str.split(',')]
    try:
        input_interests = data_dict['mlb_users'].transform([interests_list])
    except ValueError:
        valid_interests = [i for i in interests_list if i in data_dict['mlb_users'].classes_]
        input_interests = data_dict['mlb_users'].transform([valid_interests])

    input_demo = pd.DataFrame({'budget': [budget_str], 'age_group': [age_str]})
    input_demo_vec = data_dict['ohe'].transform(input_demo)

    input_vector = np.hstack([input_interests, input_demo_vec])
    input_tensor = torch.tensor(input_vector, dtype=torch.float32)

    generator.eval()
    with torch.no_grad():
        generated_embedding = generator(input_tensor)

    place_embs_np = data_dict['place_embeddings'].numpy()
    gen_emb_np = generated_embedding.numpy()

    scores = cosine_similarity(gen_emb_np, place_embs_np)[0]

    top_indices = scores.argsort()[-5:][::-1]

    print("\nÖnerilen Mekanlar:")
    for idx in top_indices:
        place = data_dict['places_df'].iloc[idx]
        score = scores[idx]
        print(f"- {place['name']} ({place['category']}) | Skor: {score:.3f}")

# =============================================================================
# ÇALIŞTIRMA KISMI
# =============================================================================

# 1. Veriyi Hazırla
data = load_and_process_data()

# 2. Modeli Eğit
trained_gen = train_coldgan(data, epochs=200)

# 3. MANUEL TEST - KENDİ DENEMELERİNİ BURADAN YAPABİLİRSİN
# Örnek 1: Alışveriş seven (CSV'de 'shopping' var), Lüks bütçeli
test_new_user(
    trained_gen,
    data,
    interests_str="shopping",
    budget_str="Luxury",
    age_str="25-34"
)

# Örnek 2: Park ve doğa seven
test_new_user(
    trained_gen,
    data,
    interests_str="parks, nature",
    budget_str="Standard",
    age_str="55"
)